# 1. Extract

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import os
import glob
import time
from io import StringIO
import shutil
import requests as req
from bs4 import BeautifulSoup as bs 
import unicodedata
from functools import reduce

## 1.1 Unemployment
- Portal datos Abiertos Gobierno España
     - Direct download

In [113]:
def read_csv_unemployment(folder_path, file_name):
    """
    Reads a CSV file from a folder and creates a pandas DataFrame, corrects the unicode errors and overwrite the csv

    Args:
        folder_path (str): The path to the folder containing the CSV file.
        file_name (str): The name of the CSV file.

    """
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path ,encoding='utf-8')
    
    return df

In [115]:
df=read_csv_from_folder('/app/data','Paro_por_municipios_2019.csv')

In [116]:
df.head()

,Código mes,mes,Código de CA,Comunidad Autónoma,Codigo Provincia,Provincia,Codigo Municipio,Municipio,total Paro Registrado,Paro hombre edad < 25,Paro hombre edad 25 -45,Paro hombre edad >=45,Paro mujer edad < 25,Paro mujer edad 25 -45,Paro mujer edad >=45,Paro Agricultura,Paro Industria,Paro Construcción,Paro Servicios,Paro Sin empleo Anterior
0,201901,Enero de 2019,1,Andalucía,4,Almería,4001,Abla,77,4,15,23,4,21,10,11,5,5,52,4
1,201901,Enero de 2019,1,Andalucía,4,Almería,4002,Abrucena,62,5,9,14,5,16,13,7,6,9,36,4
2,201901,Enero de 2019,1,Andalucía,4,Almería,4003,Adra,2088,103,385,449,97,569,485,386,73,250,1174,205
3,201901,Enero de 2019,1,Andalucía,4,Almería,4004,Albánchez,18,0,2,6,1,7,2,1,0,2,15,0
4,201901,Enero de 2019,1,Andalucía,4,Almería,4005,Alboloduy,35,1,8,12,1,8,5,4,4,7,19,1


----------------------------

## 1.2 Income Tax
- BeaufitulSoup

In [25]:
url_scraping= "https://www.agenciatributaria.es/AEAT/Contenidos_Comunes/La_Agencia_Tributaria/Estadisticas/Publicaciones/sites/irpfmunicipios/2019/jrubikf74b3dca9af01b51cabd6d5603e0e16daecd1a97c.html"

In [26]:
html=req.get(url_scraping).content

soup=bs(html, 'html.parser')

tabla=soup.find_all('table')  # find_all(tag)

In [27]:
columnas=[e.text.strip() for e in tabla[0].find_all('th')]

###### Get namecols "RENTA BRUTA MEDIA , RENTA DISPONIBLE MEDIA'

In [28]:
cols = columnas[6:8]
cols_ccaa = [item.replace('RENTA BRUTA MEDIA','renta_bruta_media').replace('RENTA DISPONIBLEMEDIA', 'renta_disponible_media') for item in cols]
cols_ccaa.insert(0, "ccaa")
cols_provincias=[item.replace('RENTA BRUTA MEDIA','renta_bruta_media').replace('RENTA DISPONIBLEMEDIA', 'renta_disponible_media') for item in cols]
cols_provincias.insert(0, "provincia")

###### Get CCAA Data (except Pais Vasco and Navarra)

In [29]:
ccaa_renta_bruta_media = []
ccaa_renta_disponible_media = []
ccaa=[]

In [30]:
for table in soup.find_all("tr", class_="depth_1"):
    ccaa.append(table.find_all('th')[0].text)
    rb=table.find('td', {'headers': 'table01_c6'})
    ccaa_renta_bruta_media.append(rb.text)
    rm=table.find('td', {'headers': 'table01_c7'})
    ccaa_renta_disponible_media.append(rm.text)

In [31]:
#Erase characters
ccaa=[el.replace(u'\xa0','') for el in ccaa]
ccaa=[el.strip() for el in ccaa]

In [32]:
### Get data from provinces
provinces_table = soup.findAll(class_= 'depth_2') 
provincias_renta_bruta_media = []
provincias_renta_disponible_media = []
provincias=[]

In [33]:
for table in soup.find_all('tr', class_='depth_2'):
    provincias.append(table.find_all('th')[0].text)
    provincias_renta_bruta_media.append(table.find('td', {'headers': 'table01_c6'}).text)
    provincias_renta_disponible_media.append(table.find('td', {'headers': 'table01_c7'}).text)

In [34]:
#Erase characters
provincias=[el.replace(u'\xa0','') for el in provincias]
provincias=[el.strip() for el in provincias]

#### Get Pais Vasco data

In [35]:
url_PV = 'https://www.eustat.eus/elementos/ele0005700/renta-personal-media-de-la-c-a-de-euskadi-por-ambitos-territoriales-segun-tipo-de-renta-euros/tbl0005790_c.html'

In [36]:
html=req.get(url_PV).content

soup=bs(html, 'html.parser')

tabla_e=soup.find_all('table')

In [37]:
for i,table in enumerate(soup.find_all("tr", class_="Nivel3B")):
    if i==0:
        ccaa.append(table.find('td').text)
        ccaa_renta_bruta_media.append(table.find('td', {'class': 'td8Bold'}).text)
        ccaa_renta_disponible_media.append(table.find_all('td', {'class': 'td8Bold'})[-1].text)

    elif i==1:
        provincias.append(table.find('td').text)
        provincias_renta_bruta_media.append(table.find('td', {'class': 'td8'}).text)
        provincias_renta_disponible_media.append(table.find_all('td', {'class': 'td8'})[-1].text)

    elif i==2:
        provincias.append(table.find('td').text)
        provincias_renta_bruta_media.append(table.find('td', {'class': 'td8'}).text)
        provincias_renta_disponible_media.append(table.find_all('td', {'class': 'td8'})[-1].text)
    
    elif i==3:
        provincias.append(table.find('td').text)
        provincias_renta_bruta_media.append(table.find('td', {'class': 'td8'}).text)
        provincias_renta_disponible_media.append(table.find_all('td', {'class': 'td8'})[-1].text)

In [38]:
ccaa[-1]='Pais Vasco'
ccaa=[el.replace(u'\xa0','') for el in ccaa]
ccaa=[el.strip() for el in ccaa]
provincias=[el.replace(u'\xa0','') for el in provincias]
provincias=[el.strip() for el in provincias]

#### Get Navarra data

In [39]:
ccaa.insert(20,'Navarra')
ccaa_renta_disponible_media.append(13.394)
ccaa_renta_bruta_media.append(16.300)

In [40]:
provincias.insert(53,'Navarra')
provincias_renta_bruta_media.append(13.394)
provincias_renta_disponible_media.append(16.300)

------------------------------------------------

## OUTPUT DATA

In [41]:
df_ccaa_renta = pd.DataFrame(zip(ccaa, ccaa_renta_bruta_media,ccaa_renta_disponible_media),
               columns =[cols_ccaa])

In [42]:
df_ccaa_renta.head()

,ccaa,renta_bruta_media,renta_disponible_media
0,Andalucía,23.510,19.682
1,Aragón,27.014,22.200
2,"Asturias, Principado de",27.448,22.674
3,"Balears, Illes",29.736,24.182
4,Canarias,25.087,20.938


In [43]:
df_provincias_renta = pd.DataFrame(zip(provincias, provincias_renta_bruta_media,provincias_renta_disponible_media),
               columns =[cols_provincias])

In [44]:
df_provincias_renta.head()

,provincia,renta_bruta_media,renta_disponible_media
0,Almería,21.399,18.062
1,Cádiz,24.497,20.538
2,Córdoba,21.879,18.432
3,Granada,23.515,19.737
4,Huelva,20.972,17.717


In [49]:
def export_to_csv(df, file_path):
    
    df.to_csv(file_path, encoding='latin-1', index=False, sep=';')
    
    return print(f"File saved to {file_path}")

In [50]:
export_to_csv(df_ccaa_renta,'/app/data/csv/ccaa_renta.csv')

File saved to /app/data/csv/ccaa_renta.csv


In [51]:
export_to_csv(df_provincias_renta,'/app/data/csv/provincias_renta.csv')

File saved to /app/data/csv/provincias_renta.csv
